In [ ]:
import pandas as pd
import cPickle as pickle
from IPython.display import display
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline

In [ ]:
remap_dict = {
    ('unmeasure', 'blur'): 'unmeasure-blur',
    ('unmeasure', 'inpaint-tv') : 'unmeasure-inpaint-tv',
    ('baseline', 'None') : 'ignore',
    ('ambient', 'None') : 'AmbientGAN (ours)',
    ('unmeasure', 'wiener'): 'unmeasure-weiner',
}

In [ ]:
def convert_df(df, convert_spec):
    for colname in convert_spec['float']:
        df.loc[:, colname] = df[colname].apply(np.float32)
    return df

In [ ]:
def filter_df(df, filter_spec):
    df = df[filter_spec['cols']]
    filter_ = True
    for colname, values in filter_spec['in'].iteritems():
        filter_ = filter_ & df[colname].isin(values)
    df = df[filter_]
    return df

In [ ]:
def save_plot(save_path):
    pdf = PdfPages(save_path)
    pdf.savefig(bbox_inches='tight')
    pdf.close()    

def errorbar(x, y, std, **kwargs):
    (_, caps, _) = plt.errorbar(x, y, yerr=1.96*std,
                                marker='o', markersize=5, capsize=5, linewidth=2, **kwargs)
    for cap in caps:
        cap.set_markeredgewidth(1)

In [ ]:
def plot1(plot_spec, df):

    figure_groups = df.groupby(plot_spec['figure_group_by'])
    for fig_key, fig_val in figure_groups.groups.iteritems():
        fig_group_df = df.loc[fig_val, :]

        plt.figure(figsize=[6, 4])
        # plt.title(fig_key)
        
        curve_groups = fig_group_df.groupby(plot_spec['curve_group_by'])
        x_var = plot_spec['x_var']
        y_var = plot_spec['y_var']
        std_var = plot_spec['std_var']
        
        legends = []

        for curve_key in sorted(curve_groups.groups):
            curve_val = curve_groups.groups[curve_key]
            # for curve_key, curve_val in curve_groups.groups.iteritems():
            group_df = fig_group_df.loc[curve_val, :]
            legends.append(remap_dict[curve_key])

            cols = [x_var, y_var, std_var]
            group_df.sort_values(by=[x_var], inplace=True)
            x = np.array(group_df[x_var])
            y = np.array(group_df[y_var])

            if std_var is not None:
                std = np.array(group_df[std_var])
                errorbar(x, y, std, alpha=0.8)
            else:
                plt.plot(x, y, '-o')

        ## Prettify
        # axis
        plt.gca().set_ylim(bottom=0)
        # plt.gca().set_xscale("log", nonposx='clip')
        # plt.gca().set_xlim([-0.1, 1.1])

        # labels, ticks, titles
        # ticks = [10, 25, 50, 100, 200, 300, 400, 500, 750]
        # labels = [10, 25, 50, 100, 200, 300, 400, 500, 750]
        # plt.xticks(ticks, labels, rotation=90)
        plt.xlabel(plot_spec['x_label'], fontsize=14)
        plt.ylabel(plot_spec['y_label'], fontsize=14)

        # Legends
        plt.legend(legends, fontsize=12.5, loc=plot_spec['legend_loc'])
        plt.ylim(plot_spec['y_lim'])

        if plot_spec['save']:
            figname = '_'.join(fig_key)
            fig_savepath = './results/plots/{}.pdf'.format(figname)
            save_plot(fig_savepath)
               
def plot2(df):
    fig = plt.figure(figsize=[6, 4])
    ax = plt.subplot(111)
    legends = []
    legend_dict = {'ambient': 'AmbientGAN', 'baseline': 'ignore', 'unmeasure': 'unmeasure-blur'}
    colors = {'ambient': 'blue', 'baseline': 'green', 'unmeasure': 'red'}
    df.sort_values(by=['train_mode', 'drop_prob'], inplace=True)
    for i, row in df.iterrows():
        drop_prob = np.around(row['drop_prob'], decimals=2)
        train_mode = row['train_mode']
        legend = 'p = {}, {}'.format(legend_dict[train_mode], drop_prob)
        legends.append(legend)
        iters, scores = zip(*row['m_inception'])
        mean, std = map(np.array, zip(*scores))
        errorbar(iters, mean, std, alpha=1-drop_prob/1.1, color=colors[train_mode])
    # plt.legend(legends, fontsize=12.5, loc=2)
    plt.xlabel('Training iteration',  fontsize=14)
    plt.ylabel('Inception score', fontsize=14)
    
    #     # Following block is based on https://stackoverflow.com/a/4701285/3537687
    #     # Shrink current axis by 20%
    #     box = ax.get_position()
    #     ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    #     # Put a legend to the right of the current axis
    #     ax.legend(legends, loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.ylim([0, 8.2])
    
    fig_savepath = './results/plots/cifar10_drop_independent_vs-time.pdf'
    save_plot(fig_savepath)

In [ ]:
def produce_plot1(df_filepath, convert_spec, filter_spec, plot_spec):
    with open(df_filepath, 'rb') as df_pkl_file:
        df = pickle.load(df_pkl_file)
    
    df = convert_df(df, convert_spec)    
    df = filter_df(df, filter_spec)    
    display(df)
    plot1(plot_spec, df)
    
def produce_plot2(df_filepath, convert_spec, filter_spec):
    with open(df_filepath, 'rb') as df_pkl_file:
        df = pickle.load(df_pkl_file)
    df = convert_df(df, convert_spec)    
    df = filter_df(df, filter_spec)    
    #     display(df)
    plot2(df)

In [ ]:
def main1():
    df_filepath = './results/df_cifar10.pkl'

    convert_spec = {
        'float': [
            'drop_prob',
        ]        
    }

    filter_spec = {

        'cols' : [
            'dataset', 
            'measurement_type', 
            'train_mode', 
            'drop_prob',
            'unmeasure_type', 
            # 'm_inception',
            'm_final_score',
            'm_final_std',
        ],

        'in': {
            'measurement_type': [
                'drop_independent'
            ]
        }

    }

    plot_spec = {
        'figure_group_by' : ['dataset', 'measurement_type'],    
        'curve_group_by': ['train_mode', 'unmeasure_type'],
        # change these to lambda functions if needed
        'x_var': 'drop_prob',
        'y_var': 'm_final_score',
        'std_var': 'm_final_std',
        'x_label': 'Block probability (p)',
        'y_label': 'Inception score',
        'save': True,
        'y_lim': [0, 8.2],
        'legend_loc': 3,
    }

    produce_plot1(df_filepath, convert_spec, filter_spec, plot_spec)
    
main1()

In [ ]:
def main3():
    df_filepath = './results/df_cifar10.pkl'

    convert_spec = {
        'float': [
            'drop_prob',
            'additive_noise_std',
        ]        
    }

    filter_spec = {

        'cols' : [
            'dataset', 
            'measurement_type', 
            'train_mode', 
            'additive_noise_std',
            'unmeasure_type', 
            # 'm_inception',
            'm_final_score',
            'm_final_std',
        ],

        'in': {
            'measurement_type': [
                'blur_addnoise'
            ]
        }

    }

    plot_spec = {
        'figure_group_by' : ['dataset', 'measurement_type'],    
        'curve_group_by': ['train_mode', 'unmeasure_type'],
        # change these to lambda functions if needed
        'x_var': 'additive_noise_std',
        'y_var': 'm_final_score',
        'std_var': 'm_final_std',
        'x_label': 'Noise standard deviation ($\sigma$)',
        'y_label': 'Inception score',
        'save': True,
        'y_lim': [0, None],
        'legend_loc': 1,
    }

    produce_plot1(df_filepath, convert_spec, filter_spec, plot_spec)
    
main3()

In [ ]:
def main2():
    df_filepath = './results/df_cifar10.pkl'

    convert_spec = {
        'float': [
            'drop_prob',
        ]        
    }

    filter_spec = {

        'cols' : [
            'dataset', 
            'measurement_type', 
            'train_mode', 
            'drop_prob',
            'unmeasure_type', 
            'm_inception',
        ],

        'in': {
            'measurement_type': [
                'drop_independent'
            ],
#             'train_mode': ['ambient', 'baseline']
        }

    }
    
    produce_plot2(df_filepath, convert_spec, filter_spec)
    
main2()